In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import tensorflow as tf

Import Data Set

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


Preprocessing the Data

In [5]:
from tensorflow.keras.layers import TextVectorization

In [6]:
x=df['comment_text']
y=df.iloc[:,2:].values

In [7]:
x.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [8]:
from tensorflow.keras.layers import TextVectorization

In [9]:
MAX_VALUE=200000

In [10]:
vectorizer=TextVectorization(max_tokens=MAX_VALUE, output_sequence_length=1800, output_mode='int')

In [11]:
vectorizer.adapt(x.values)

In [12]:
TextVectorization

keras.layers.preprocessing.text_vectorization.TextVectorization

In [13]:
vectorized_text=vectorizer(x.values)

In [14]:
#MCSHBAP --> MAP, CHACHE, SHUFFLE, BATCH, PREFETCH from_tensor_slices, list_dir
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))

In [15]:
dataset=dataset.cache()

In [16]:
dataset=dataset.shuffle(160000)

In [17]:
dataset.batch(16)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>

In [18]:
dataset.prefetch(8)

<_PrefetchDataset element_spec=(TensorSpec(shape=(1800,), dtype=tf.int64, name=None), TensorSpec(shape=(6,), dtype=tf.int64, name=None))>

In [19]:
train=dataset.take(int(len(dataset)*.7))

In [20]:
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))

In [21]:
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

Creating a Deep Neural Network

In [22]:
from tensorflow.keras.models import Sequential

In [23]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Embedding

In [55]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_VALUE+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [56]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dense_9 (Dense)             (None, 256)               33024     
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 6)                 774       
                                                      

In [60]:
history = model.fit(train, epochs=1, validation_data=val)

ValueError: in user code:

    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    ValueError: Exceptio

Training the model

Make Predictions

In [47]:
input_text=vectorizer("You freaking suck! I am going to hurt you")

In [48]:
input_text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([   7, 7158,  397, ...,    0,    0,    0], dtype=int64)>

In [49]:
df.iloc[:,2:]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [50]:
model.predict(np.array([input_text]))

1/1 [==============================] - 0s 474ms/step


array([[0.50188965, 0.51303905, 0.49233615, 0.49876395, 0.50449073,
        0.49476725]], dtype=float32)

Evaluate the Model

In [51]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [52]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [53]:
for batch in test.as_numpy_iterator(): 
    X_true, y_true = batch
    yhat = model.predict(X_true)
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

TypeError: in user code:

    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\kabir\anaconda3\lib\site-packages\keras\layers\rnn\lstm.py", line 615, in call
        timesteps = input_shape[0] if self.time_major else input_shape[